In [ ]:
!pip install kaggle-environments --upgrade
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *
import random
import numpy as np
from scipy.optimize import linear_sum_assignment
# Create a test environment for use later
environment = make("halite", configuration={"size": 10, "startingHalite": 1000}, debug=True)

In [ ]:
agent_count = 2
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)
print(board)

In [ ]:
# helpers
def dist(a: Point, b: Point) -> int:
    return min(abs(a.x-b.x), BOARD_SIZE-abs(a.x-b.x)) + min(abs(a.y-b.y), BOARD_SIZE-abs(a.y-b.y))

def direction_to(s: Point, t: Point) -> ShipAction:
    N = environment.configuration.size
    candidate = [] # [N/S, E/W]
    if s.x-t.x != 0:
        candidate.append(ShipAction.WEST if (s.x-t.x) % N < (t.x-s.x) % N else ShipAction.EAST)
    if s.y-t.y != 0:
        candidate.append(ShipAction.SOUTH if (s.y-t.y) % N < (t.y-s.y) % N else ShipAction.NORTH)
    return random.choice(candidate) if len(candidate) > 0 else None

def sort_cells_with_halite(board):
    return sorted([cell for cell in board.cells.values()], key=lambda cell : cell.halite, reverse=True)

def 

In [ ]:

    
def cost(ship, cell):
    # TODO: much to improve
    cfg = environment.configuration
    haliteCoef = cfg.size / cfg.maxCellHalite
    return dist(ship.position, cell.position) - haliteCoef * cell.halite

@board_agent
def agent(board):
    # mine
    miningCells = sort_cells_with_halite(board)
    ships = board.current_player.ships
    shipyards = board.current_player.shipyards
    costMatrix = np.array([[cost(ship, cell) for ship in ships] for cell in miningCells])
    tasks, _ = linear_sum_assignment(costMatrix)
    
    for i, ship in enumerate(ships):
        ship.next_action = direction_to(ship.position, miningCells[tasks[i]].position)
    if len(shipyards) == 0:
        ships[0].next_action = ShipAction.CONVERT
    for shipyard in shipyards:
        if shipyard.cell.ship is None:
            shipyard.next_action = ShipyardAction.SPAWN

In [ ]:
environment.reset(agent_count)
environment.run([agent, "random"])
environment.render(mode="ipython", width=500, height=450)

In [ ]:
environment.configuration